## Quickstart and check llm connectivity

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import requests

load_dotenv()  # Load environment variables from .env file

True

In [3]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn."
        }
    ]
)

print(completion.choices[0].message.content)

Once upon a time, a majestic unicorn danced under the moonlight, spreading magic and dreams throughout the land.


## Actual function calling experiments

In [4]:
# Define the tools for function calling

# write a function to get current weather
def get_current_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


# write a function to send email
def send_email(recipients, subject, body):
    # This is a placeholder function. In a real application, you would use an email sending service.
    print(f"Sending email to: {', '.join(recipients)}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    return {"status": "success", "message": "Email sent successfully."}


# write a function to query knowledge base
def query_knowledge_base(query, limit):
    # This is a placeholder function. In a real application, you would query a database or an API.
    print(f"Querying knowledge base for: {query} with limit {limit}")
    return {"results": [{"title": "ChatGPT Overview", "content": "ChatGPT is a language model developed by OpenAI."}] * limit}  # Simulating multiple results for the sake of example

In [5]:
# Define the tools for the OpenAI client

# get_current_weather_function_tool
get_current_weather_function_tool = {
  "name": "get_current_weather",
  "description": "Get current weather in a given location for provided coordinates. Default is Celsius.",
  "strict": True,
  "parameters": {
    "type": "object",
    "required": [
      "latitude",
      "longitude",
      "unit"
    ],
    "properties": {
      "latitude": {
        "type": "number",
        "description": "Latitude coordinate of the location"
      },
      "longitude": {
        "type": "number",
        "description": "Longitude coordinate of the location"
      },
      "unit": {
        "type": "string",
        "description": "Unit of temperature measurement",
        "enum": [
          "Celsius",
          "Fahrenheit"
        ]
      }
    },
    "additionalProperties": False
  }
}

# send_email_function_tool
send_email_function_tool = {
    "name": "send_email",
    "description": "Send an email to one or more recipients.",
    "strict": True,
    "parameters": {
        "type": "object",
        "required": ["recipients", "subject", "body"],
        "properties": {
            "recipients": {
                "type": "array",
                "items": {
                    "type": "string",
                    "format": "email"
                },
                "description": "List of email addresses to send the email to."
            },
            "subject": {
                "type": "string",
                "description": "Subject line of the email."
            },
            "body": {
                "type": "string",
                "description": "Body content of the email."
            }
        },
        "additionalProperties": False
    }
}

# query_knowledge_base_function_tool
query_knowledge_base_function_tool = {
  "name": "query_knowledge_base",
  "description": "Query a knowledge base to retrieve relevant info on a topic.",
  "strict": True,
  "parameters": {
    "type": "object",
    "required": [
      "query",
      "limit",
    ],
    "properties": {
      "query": {
        "type": "string",
        "description": "The search term or phrase to find in the knowledge base."
      },
      "limit": {
        "type": "number",
        "description": "Maximum number of results to return."
      },
    },
    "additionalProperties": False
  }
}

In [6]:
tools = [
    {
        "type": "function",
        "function": get_current_weather_function_tool
    },
    {
        "type": "function",
        "function": send_email_function_tool
        },
    {
        "type": "function",
        "function": query_knowledge_base_function_tool
    }
]

In [7]:
# # sample queries
messages=[{"role": "user", "content": "What is the weather like in Paris today?"}]
# messages=[{"role": "user", "content": "Can you send an email to ilan@example.com and katia@example.com saying hi?"}]
# messages=[{"role": "user", "content": "Can you find information about ChatGPT in the AI knowledge base?"}]

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools
)

[ChatCompletionMessageToolCall(id='call_fcGCmAtTzwWxY2f7FNysiMIW', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522,"unit":"Celsius"}', name='get_current_weather'), type='function')]
[ChatCompletionMessageToolCall(id='call_fcGCmAtTzwWxY2f7FNysiMIW', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522,"unit":"Celsius"}', name='get_current_weather'), type='function')]


In [ ]:
# call the function based on the tool call
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

ChatCompletionMessageToolCall(id='call_fcGCmAtTzwWxY2f7FNysiMIW', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522,"unit":"Celsius"}', name='get_current_weather'), type='function')
{'latitude': 48.8566, 'longitude': 2.3522, 'unit': 'Celsius'}


In [13]:
if tool_call.function.name == "get_current_weather":
    result = get_current_weather(args["latitude"], args["longitude"])
    print(result)
elif tool_call.function.name == "send_email":
    result = send_email(args["recipients"], args["subject"], args["body"])
    print(result)
elif tool_call.function.name == "query_knowledge_base":
    result = query_knowledge_base(args["query"], args["limit"])
    print(result)
else:
    print(f"Unknown function call: {tool_call.function.name}")
    result = None

18.4


In [99]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [100]:
completion_2.choices[0].message.content

'The current temperature in Paris today is approximately 15.9°C. If you need more detailed weather information such as conditions or forecasts, let me know!'